# Описание гипотезы
### Проверим предположения
1. Определить, насколько обучение сокращает время прохождения этапов игры.
2. Доказать, что успешное обучение само по себе влияет на оплату и не имеет значения то, каким этапом оно шло.
3. Проверить, насколько прозрачен процесс взаимодействия с игрой.
#### Проверять будем на данных пользователей, зарегистрированных в 2018 году (1 января 2018 - 31 декабря 2018)

Импротируем нужные библиотеки

In [54]:
import pandas as pd
import numpy as np

Загружаем данные

In [31]:
events_df = pd.read_csv('data/7_4_Events.csv', sep=',')
purchase_df = pd.read_csv('data/purchase.csv', sep=',')

## Производим предобработку данных

Выделяем пользователей зарегистрирванных в 2018 году и загружаем данные по ним в отдельные датафреймы

In [32]:
# Создаем фильтр по пользователям, зарегистрировавшихся в 2018 году
mask = (events_df['start_time'] >= '2018-01-01') & (events_df['start_time'] < '2019-01-01') & (events_df['event_type'] == 'registration')

regist_2018 = events_df[mask]['user_id'].to_list() # список пользователей зарегистрировавшихся в 2018 году 

events_2018 = events_df[events_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице events
purchase_2018 = purchase_df[purchase_df['user_id'].isin(regist_2018)].copy() # отбираем все записи с этими пользователями в таблице purchase


Преобразуем даты в полученых датафреймах

In [38]:
purchase_2018['event_datetime'] = pd.to_datetime(purchase_2018['event_datetime'])
events_2018['start_time'] = pd.to_datetime(events_2018['start_time'])

Перед объединением датафреймов, переименуем колонки идентификаторов

In [40]:
events_2018 = events_2018.rename(columns={'id':'event_id'})
purchase_2018 = purchase_2018.rename(columns={'id':'purchase_id'})

Сделаем объединенный датафрейм из событий и оплат

In [41]:
total_events_df = pd.concat([events_2018,purchase_2018],sort=False)

### Проверим 1-ю гипотезу
* Насколько обучение сокращает время прохождения этапов игры

Сформируем группы пользователей, которые проходили обучение и которые не проходили

In [90]:
# Пользователи которые начали обучение
users_tutorial_start = total_events_df[total_events_df['event_type'] == 'tutorial_start']['user_id'].unique()
# Пользователи которые прошли обучение
users_tutorial_finish = total_events_df[total_events_df['event_type'] == 'tutorial_finish']['user_id'].unique()
# Пользователт которые закончили обучение, по любому его начинали, а пользователи которые начали обучение,
# могли его не закончить, убираем из первого списка пользователей, которые не закончили обучение
users_tutorial_start = users_tutorial_start[~np.isin(users_tutorial_start, users_tutorial_finish)]
# Пользователи которые вообще не проходили обучение
users_not_tutorial = total_events_df[~total_events_df['user_id'].isin(users_tutorial_finish)]['user_id'].unique()

In [91]:
print(len(users_not_tutorial))
print(len(users_tutorial_finish))
print(len(users_tutorial_start))
print(len(total_events_df['user_id'].unique()))

9676
10250
1608
19926
